In [2]:
import ipywidgets as widgets
tube_length = widgets.BoundedFloatText(
value=1,
min=0,
max=10.0,
step=0.1,
description='tube length:',
disabled=False
)
tube_diameter = widgets.BoundedFloatText(
    value=.1,
    min=0,
    max=1.0,
    step=0.1,
    description='tube diameter:',
    disabled=False
)
nose_type = widgets.Select(
    options=['Ellipse', 'Cone', 'Haack', 'Parabole', 'Ogive'],
    value='Ellipse',
    # rows=10,
    description='Nose:',
    disabled=False
)
file = open("./fin.png", "rb")
image = file.read()
im = widgets.Image(
    value=image,
    format='png',
    height=100,
    width=200,
)
nose_length = widgets.BoundedFloatText(
    value=.2,
    min=0,
    max=1.0,
    step=0.1,
    description='nose length:',
    disabled=False
)
fins_number = widgets.IntText(
    value=4,
    description='nb of fins:',
    disabled=False
)
Cr = widgets.BoundedFloatText(
value=.2,
min=0,
max=1,
step=0.1,
description='Cr:',
disabled=False
)
Ct = widgets.BoundedFloatText(
value=.1,
min=0,
max=1,
step=0.1,
description='Ct:',
disabled=False
)
Xt = widgets.BoundedFloatText(
value=.1,
min=0,
max=1,
step=0.1,
description='Xt:',
disabled=False
)
s = widgets.BoundedFloatText(
value=.15,
min=0,
max=1,
step=0.1,
description='s:',
disabled=False
)
t = widgets.BoundedFloatText(
value=.01,
min=0,
max=.1,
step=0.001,
description='thickness:',
disabled=False
)
tube = widgets.VBox([
    tube_length,
    tube_diameter
])
nose = widgets.VBox([
    nose_type,
    nose_length
])
fins_fields = widgets.VBox([
    Cr,
    Ct,
    Xt,
    s,
    t,
    fins_number
])
fins = widgets.HBox([
    fins_fields,
    im

])
children = [tube,nose,fins]
tab = widgets.Tab(children)
tab.titles = ('tube', 'nose', 'fins')
tab


In [3]:
rocket_geometry = {"L":tube_length.value,
                   "l":nose_length.value,
                   "d":tube_diameter.value,
                   "nb_fins":fins_number.value,
                   "nose_type":nose_type.value,
                   "Cr":Cr.value,
                   "Ct":Ct.value,
                   "Xt":Xt.value,
                   "s":s.value,
                   "t":t.value}


In [4]:
from pythreejs import *
from IPython.display import display
d = rocket_geometry["d"]
L = rocket_geometry["L"]

###TUBE
cylinder = CylinderGeometry(
    radiusTop=d/2, radiusBottom=d/2, height=L, radialSegments=15, color="#ff1234"
)
myCylinder = Mesh(
    geometry = cylinder,
    material=MeshLambertMaterial(color='green',vertexColors='VertexColors'),
    position=[0, 0, 0],  
)

t_ = rocket_geometry["t"]
Cr_ = rocket_geometry["Cr"]
Ct_ = rocket_geometry["Ct"]
Xt_ = rocket_geometry["Xt"]
s_ = rocket_geometry["s"]

print(Xt_)

###FINS
vertices = [
    [-t_/2, 0, 0],
    [-t_/2, Cr_-Ct_-Xt_, s_],
    [-t_/2, Cr_, 0],
    [-t_/2, Cr_-Xt_, s_],
    [t_/2, 0, 0],
    [t_/2, Cr_-Ct_-Xt_, s_],
    [t_/2, Cr_, 0],
    [t_/2, Cr_-Xt_, s_]
]
faces = [
    [0, 1, 3],
    [0, 3, 2],
    [0, 2, 4],
    [2, 6, 4],
    [0, 4, 1],
    [1, 4, 5],
    [2, 3, 6],
    [3, 7, 6],
    [1, 5, 3],
    [3, 5, 7],
    [4, 6, 5],
    [5, 6, 7]
]
vertexcolors = ['#ffffff', '#ffffff', '#ffffff', '#ffffff',
                '#ffffff', '#ffffff', '#ffffff', '#ffffff']

# Map the vertex colors into the 'color' slot of the faces
faces = [f + [None, [vertexcolors[i] for i in f], None] for f in faces]

# Create the geometry:
cubeGeometry = Geometry(vertices=vertices,
    faces=faces,
    colors=vertexcolors)
# Calculate normals per face, for nice crisp edges:
cubeGeometry.exec_three_obj_method('computeFaceNormals')

nb_fins = rocket_geometry['nb_fins']
d = rocket_geometry['d']
L = rocket_geometry['L']

children = []

for i in range(nb_fins):
    globals()[f"myFin{i}"] = Mesh(
    geometry=cubeGeometry,
    material=MeshLambertMaterial(color='blue',vertexColors='VertexColors'),
    position=[d/2 * np.sin(np.pi/(nb_fins/2) * i), -L/2, d/2 * np.cos(np.pi/(nb_fins/2) * i)],   
)
    children.append(globals()[f"myFin{i}"])
    globals()[f"myFin{i}"].rotateY(np.pi/(nb_fins/2) * i)


###NOSE

def nose_tangent_ogive(x, R, L):
    rho = (R**2 + L**2)/(2*R)
    return np.sqrt(rho**2 - (L-x)**2) + R - rho 

def nose_ellipse(x,R,L):
    return R*np.sqrt(1-x**2/L**2)

def nose_parabole(x, R, L, K):
    #K entre 0 et 1 compris
    return R*((2*(x/L)-K*(x/L)**2)/(2-K))

def nose_power_series(x,R,L,n):
    #n entre 0 et 1
    return R*(x/L)**n 

def nose_hack_series(x,R,L,C):
    #si C=1/3, serie LV-Haack, minimum drag, si C=2/3, tangent to the body at the base
    theta = np.arccos(1-2*x/L)
    return R/np.sqrt(np.pi) * np.sqrt(theta- np.sin(2*theta)/2 + C*np.sin(theta)**3)

def nose_cone(x,R,L):
    return x*R/L

l=rocket_geometry["l"]
x=np.linspace(0,l,20)
nosetype = rocket_geometry["nose_type"]



nose_points = []
for i in range(len(x)):
    if nosetype == "Ellipse":
        nose_points.append([nose_ellipse(x[i],d/2,l), x[i], 0])
    elif nosetype == "Parabole":
        nose_points.append([nose_parabole(x[i],d/2,l, .5), x[i], 0])
    elif nosetype == "Cone":
        nose_points.append([nose_cone(x[i],d/2,l), x[i]],0)
    elif nosetype == "Haack":
        nose_points.append([nose_hack_series(x[i],d/2,l,1/3), x[i], 0])
    else:
        nose_points.append([nose_tangent_ogive(x[i],d/2,l), x[i], 0])

nose = LatheGeometry(
    points=nose_points,
    segments=16,
    phiStart=0.0,
    phiLength=2.0*pi, _flat=True)

myNose = Mesh(
    geometry = nose,
    material=MeshLambertMaterial(color='red',vertexColors='VertexColors'),
    position=[0, L/2 + l if nosetype!="Ellipse" else L/2, 0],   
)
if nosetype!="Ellipse":
    myNose.rotateZ(np.pi)


fins = Group(children=children)
corps= Group(children=[myCylinder,myNose,fins])

l = LightShadow(camera=UninitializedSentinel)

axeshelper = AxesHelper(3)

# Set up a scene and render it:
cCube = PerspectiveCamera(position=[10, 5, 10], fov=10,
                    children=[DirectionalLight(color='#ffffff', position=[-3, 5, 1], intensity=0.5, target=corps)])
sceneCube = Scene(children=[corps, axeshelper, AmbientLight(color='#dddddd')], background=None)

rendererCube = Renderer(camera=cCube, background='red', background_opacity=1,
                        scene=sceneCube, controls=[OrbitControls(controlling=cCube, autoRotate=True)], width = 500, height = 500)
display(rendererCube)


0.1


Renderer(background='red', camera=PerspectiveCamera(children=(DirectionalLight(intensity=0.5, position=(-3.0, …

In [5]:
spin_track = NumberKeyframeTrack(name='.rotation[y]', times=[0, 2], values=[0, 6.28])
spin_clip = AnimationClip(tracks=[spin_track])
spin_action = AnimationAction(AnimationMixer(corps), spin_clip, corps)
spin_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[y]', times=array([0, 2]), value…